# Predict on new data using a trained CNN on XPS data on Google Colab

In this notebook, we will use a trained convolutional network to predict on unseen XPS spectra.

## Setup

### Mount google drive, change working directory

In [ ]:
# Mount drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Change working path
os.chdir('/content/drive/My Drive/deepxps')

### Install packages and import modules

In [ ]:
%%capture
# Install packages
!pip install python-docx
!pip install xlsxwriter
# Import standard modules and magic commands
import datetime
import numpy as np
import pytz
import importlib
import matplotlib.pyplot as plt

# Magic commands
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Disable tf warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf

### Set seeds and restart session to ensure reproducibility

In [ ]:
def reset_seeds_and_session(seed=1):
   os.environ['PYTHONHASHSEED']=str(seed)
   tf.random.set_seed(seed)
   np.random.seed(seed)

   session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                                           inter_op_parallelism_threads=1)
   sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),
                               config=session_conf)
   tf.compat.v1.keras.backend.set_session(sess) 

reset_seeds_and_session(seed=1)

### Check TensorFlow version

In [ ]:
f"TF version: {tf.__version__}."

### Load custom modules

In [ ]:
try:
    import importlib
    importlib.reload(classifier)
    importlib.reload(clfutils)
    print('\n Modules were reloaded.')
except:
    import xpsdeeplearning.network.classifier as classifier
    import xpsdeeplearning.network.utils as clfutils
    print('Modules were loaded.')

### Set up the parameters & folder structure

In [ ]:
time = datetime.datetime.now().astimezone(pytz.timezone('Europe/Berlin')).strftime("%Y%m%d_%Hh%Mm")
exp_name = "Ni_2_classes_MC_dropout_test_predict_using_20220224_16h08m"

clf = classifier.Classifier(time = time,
                            exp_name = exp_name,
                            task = "regression",
                            intensity_only = True)

### Load and inspect the data

In [ ]:
input_filepath = r'/content/drive/My Drive/deepxps/datasets/20210528_Ni_linear_combination_small_gas_phase.h5'

train_test_split = 0.99
train_val_split = 0
no_of_examples = 500

        
X_train, X_val, X_test, y_train, y_val, y_test,\
    sim_values_train, sim_values_val, sim_values_test =\
        clf.load_data_preprocess(input_filepath = input_filepath,
                                 no_of_examples = no_of_examples,
                                 train_test_split = train_test_split,
                                 train_val_split = train_val_split,
                                 select_random_subset=False,
                                 shuffle=False,
                                 )
        
# Check how the examples are distributed across the classes.
class_distribution = clf.datahandler.check_class_distribution(clf.task)
clf.plot_class_distribution()
clf.plot_random(no_of_spectra = 10, dataset = 'test')  

### Load and compile the model

In [ ]:
from tensorflow.python.keras import backend as K
clf.load_model(model_path = '/content/drive/My Drive/deepxps/runs/20220224_16h08m_Ni_2_classes_linear_comb_regression_MC_dropout_rate02_mse/model')

### Plot summary and save model plot.


In [ ]:
clf.summary()
clf.save_and_print_model_image()

## Predict on current data set

### Evaluate on test data

In [ ]:
clf.logging.hyperparams['batch_size'] = 32

if clf.task == 'classification':
    score = clf.evaluate()
    test_loss, test_accuracy = score[0], score[1]
    print('Test loss: ' + str(np.round(test_loss, decimals=8)))
    print('Test accuracy: ' + str(np.round(test_accuracy, decimals=3)))
elif clf.task == 'regression':
    test_loss = clf.evaluate()
    print('Test loss: ' + str(np.round(test_loss, decimals=8)))

### Show predictions with current model

In [ ]:
no_of_predictions = 1000

prob_pred_train = clf.predict_probabilistic(
    dataset="train",
    no_of_predictions=no_of_predictions
)
prob_pred_test = clf.predict_probabilistic(
    dataset="test",
    no_of_predictions=no_of_predictions
)

for i, pred in enumerate(prob_pred_test[:10]):
   print(f"Ground truth: {np.round(clf.datahandler.y_test[i],3)},",
         f"Mean prediction: {np.mean(pred, axis = 0)} +/- {np.std(pred, axis = 0)}")

### Show some predictions on random test samples

In [ ]:
clf.plot_prob_predictions(
    dataset="test",
    kind="random",
    no_of_spectra=20,
    to_file=True)

### Show predictions with highest standard deviation

In [ ]:
clf.plot_prob_predictions(
    dataset="test",
    kind="max",
    no_of_spectra=20,
    to_file=True)

### Show predictions with lowest standard deviation

In [ ]:
clf.plot_prob_predictions(
    dataset="test",
    kind="min",
    no_of_spectra=20,
    to_file=True)

## Create out-of-scope dataset

### Plot specific spectra and probabilistic predictions

In [ ]:
#indices = [129, 281, 357, 468, 50, 227, 237, 29, 230]
indices = [209, 29]

clf.plot_spectra_by_indices(
    indices=indices, 
    dataset="test",
    with_prediction=True)

In [ ]:
fig = clf.datahandler.plot_prob_predictions(
    dataset="test",
    prob_preds=clf.datahandler.prob_pred_test,
    indices=indices, 
    no_of_spectra=len(indices),
    #to_file=True
    )

In [ ]:
for i in indices:
    print(i, clf.datahandler.sim_values_test["shift_x"][i])

### Create synthetic spectra

In [ ]:
from xpsdeeplearning.simulation.base_model.spectra import SyntheticSpectrum, SimulatedSpectrum
from xpsdeeplearning.simulation.base_model.peaks import Gauss

class SyntheticSpectraWrapper():
    def __init__(self, base_spectrum_index):
        self.base_spectrum_index = base_spectrum_index
        self.energies = clf.datahandler.energies
        self.step = np.round(self.energies[0] - self.energies[1], 2)
        self.syn_spectra = []

        self.base_spectrum = SimulatedSpectrum(self.energies[-1],
            self.energies[0],
            self.step,
            label="synthetic"
            )
        self.base_spectrum.lineshape = clf.datahandler.X_test[base_spectrum_index].squeeze()
        shift = -clf.datahandler.sim_values_test["shift_x"][base_spectrum_index]
        self.base_spectrum.shift_horizontal(shift)

        self.X_template = self.base_spectrum.lineshape[:, None]

    def get_spectra_stack(self):
        lineshapes = [spectrum.lineshape for spectrum in self.syn_spectra]
        return np.hstack(lineshapes)[None, :, :].transpose(2, 1, 0)
          
    def _normalize_min_max(self, data):
        return (data - np.min(data)) / (np.max(data) - np.min(data))

    def add_peaks_to_spectrum(self,peak_params):
        syn_spectrum = SyntheticSpectrum(
            self.energies[-1],
            self.energies[0],
            self.step,
            label="synthetic"
            )
        syn_spectrum.add_component(
            Gauss(position=peak_params["positions"][0],
                  width=peak_params["widths"][0],
                  intensity=peak_params["intensities"][0]))
        syn_spectrum.add_component(
            Gauss(position=peak_params["positions"][1],
                  width=peak_params["widths"][1],
                  intensity=peak_params["intensities"][1]))
        syn_spectrum.lineshape += self.base_spectrum.lineshape
        syn_spectrum.lineshape += np.min(self.base_spectrum.lineshape)
        syn_spectrum.lineshape = syn_spectrum.lineshape[:, None]  
        syn_spectrum.normalize()

        syn_spectrum.descr = f"Test spectrum no. {self.base_spectrum_index} with \n two additional peaks (w = {peak_params['widths'][0]} eV)"

        self.syn_spectra.append(syn_spectrum)  

    def create_flat_line(self):
        syn_spectrum = SyntheticSpectrum(
            self.energies[-1],
            self.energies[0],
            self.step,
            label="flat"
        )
        syn_spectrum.lineshape += np.min(self.X_template)
        syn_spectrum.normalize()
        syn_spectrum.lineshape = syn_spectrum.lineshape[:, None]  

        syn_spectrum.descr = f"Flat line"

        self.syn_spectra.append(syn_spectrum)

    def create_synthetic_peaks(self, peak_params):
        syn_spectrum = SyntheticSpectrum(
            self.energies[-1],
            self.energies[0],
            self.step,
            label="peaks"
            )    
        syn_spectrum.add_component(
            Gauss(position=peak_params["positions"][0],
                  width=peak_params["widths"][0],
                  intensity=peak_params["intensities"][0]))
        syn_spectrum.add_component(
            Gauss(position=peak_params["positions"][1],
                  width=peak_params["widths"][1],
                  intensity=peak_params["intensities"][1]))
        syn_spectrum.lineshape += np.min(self.base_spectrum.lineshape)
        syn_spectrum.normalize()
        syn_spectrum.lineshape = syn_spectrum.lineshape[:, None]  
        syn_spectrum.normalize()

        syn_spectrum.descr = f"Broad Gaussian peaks (w = {peak_params['widths'][0]} eV)"
        
        self.syn_spectra.append(syn_spectrum)

    def create_synthetic_peaks_with_slope(self, peak_params):
        syn_spectrum = SyntheticSpectrum(
            self.energies[-1],
            self.energies[0],
            self.step,
            label="peaks_with_slope"
            )
        syn_spectrum.add_component(
            Gauss(position=peak_params["positions"][0],
                  width=peak_params["widths"][0],
                  intensity=peak_params["intensities"][0]))
        syn_spectrum.add_component(
            Gauss(position=peak_params["positions"][1],
                  width=peak_params["widths"][1],
                  intensity=peak_params["intensities"][1]))
        syn_spectrum.lineshape += np.min(self.X_template)
        syn_spectrum.normalize()
        slope = float((self.X_template[0] - self.X_template[-1]) / (self.energies[0] - self.energies[-1]))
        intercept = self.X_template[0] - slope*self.energies[0]
        line = np.array([(slope*x + intercept) for x in self.energies])
        syn_spectrum.lineshape = syn_spectrum.lineshape[:, None]  
        syn_spectrum.lineshape = np.add(syn_spectrum.lineshape, line)
        #syn_spectrum.lineshape -= np.average(self.X_template) #np.sort(self.X_template, axis=0)[:5])#np.min(self.X_template)
        syn_spectrum.normalize()

        syn_spectrum.descr = f"Broad Gaussian peaks with background slope (w = {peak_params['widths'][0]} eV)"

        self.syn_spectra.append(syn_spectrum)   
        
    def plot_spectra(self, normalize=False):
        fig, ax = plt.subplots(
            nrows=1, ncols=1, figsize=(10,10)
        )
        if normalize:
            ax.plot(self.energies, self._normalize_min_max(self.base_spectrum.lineshape))
            for lineshape in self.get_spectra_stack():
                ax.plot(self.energies, self._normalize_min_max(lineshape), linewidth=2)
        else:
            ax.plot(self.energies, self.base_spectrum.lineshape)
            for lineshape in self.get_spectra_stack():
                ax.plot(self.energies, lineshape , linewidth=2)                
        ax.invert_xaxis()
        ax.set_xlim(np.max(self.energies-clf.datahandler.sim_values_test["shift_x"][self.base_spectrum_index]), np.min(self.energies))

        fontdict = {"size": 20}
        ax.set_xlabel("Binding energy (eV)",
                        fontdict=fontdict)
        ax.set_ylabel("Intensity (arb. units)", 
                        fontdict=fontdict)
        ax.tick_params(axis="x", labelsize=fontdict["size"])
        ax.set_yticklabels([])
        ax.legend([f"Test spectrum no. {self.base_spectrum_index}"] + [s.descr for s in self.syn_spectra])

peak_params = [
    {
        "positions": [857.5, 876.5],
        "widths": [0.75, 0.75],
        "intensities": [0.0008, 0.0005]
    },
    {
        "positions": [858, 877],
        "widths": [6, 6],
        "intensities": [0.02, 0.01]
     },   

]

index = 209

spectra_wrapper = SyntheticSpectraWrapper(base_spectrum_index=index)
spectra_wrapper.add_peaks_to_spectrum(peak_params[0])
#spectra_wrapper.create_flat_line()
#spectra_wrapper.create_synthetic_peaks(peak_params[-1])
#spectra_wrapper.create_synthetic_peaks_with_slope(peak_params[-1])

spectra_wrapper.plot_spectra(normalize=True)
X_syn = spectra_wrapper.get_spectra_stack()

In [ ]:
no_of_predictions = 1000
y_syn = np.zeros([X_syn.shape[0], len(clf.datahandler.labels)])
prob_pred_syn = np.array(
    [clf.model.predict(X_syn, verbose=0) for i in range(no_of_predictions)]
    ).transpose(1, 0, 2)
print("Mean prediction:")
for i, pred in enumerate(prob_pred_syn[:10]):
    print(f"Synthetic spectrum {i}, {spectra_wrapper.syn_spectra[i].descr}: {np.mean(pred, axis = 0)} +/- {np.std(pred, axis = 0)}")

### Plot probabilistic predictions individually

In [ ]:
def plot_prob_predictions_individual(real_indices):
    no_of_real_spectra = len(real_indices)
    legend = [f"Test spectrum {i}" for i in real_indices]

    legend += [f"Synthetic spectrum {j}, {s.descr}" for j, s in enumerate(spectra_wrapper.syn_spectra)]
    
    X = np.vstack((clf.datahandler.X_test[real_indices], X_syn))
    y = np.vstack((clf.datahandler.y_test[real_indices], y_syn))
    prob_pred = np.vstack((clf.datahandler.prob_pred_test[real_indices], prob_pred_syn))

    energies = clf.datahandler.energies

    fontdict = {"size": 15}
    fig, axs = plt.subplots(
        nrows=X.shape[0], 
        ncols=len(clf.datahandler.labels)+1,
        dpi=300
        )
    plt.subplots_adjust(
        left=0.125,
        bottom=0.5,
        right=4.8,
        top=X.shape[0],
        wspace=0.4,
        hspace=0.4,
        )
    
    cmap = plt.get_cmap("tab10")

    for i in range(X.shape[0]):
        color = cmap(0) if i < no_of_real_spectra else cmap(1)

        for j, label in enumerate(clf.datahandler.labels):
            axs[i,j+1].set_title(label.split(" ")[1],fontdict=fontdict)
            axs[i,j+1].set_xlabel("Prediction",fontdict=fontdict)
            axs[i,j+1].set_ylabel("Counts",fontdict=fontdict)
            axs[i,j+1].tick_params(axis="x", labelsize=fontdict["size"])
            axs[i,j+1].tick_params(axis="y", labelsize=fontdict["size"])

        axs[i, 0].plot(energies, X[i],color=color)
        axs[i, 0].invert_xaxis()
        axs[i, 0].set_xlim(np.max(energies), np.min(energies))
        axs[i, 0].set_xlabel("Binding energy (eV)", fontdict=fontdict)
        axs[i, 0].set_ylabel("Intensity (arb. units)", fontdict=fontdict)
        axs[i,0].tick_params(axis="x", labelsize=fontdict["size"])
        axs[i,0].set_yticks([])
        axs[i,0].legend([legend[i]], fontsize=fontdict["size"]-3)

        for j, row in enumerate(prob_pred[i].transpose()):
            axs[i,j+1].hist(
                row,
                bins=100,
                range=(0.,1.),
                orientation="vertical",
                color=color,
                fill=True,
                linewidth=1,
                label=clf.datahandler.labels[j],
                )
            axs[i,j+1].set_ylim([0, prob_pred.shape[1]])

    return fig

fig = plot_prob_predictions_individual(real_indices=indices)

In [ ]:
# =============================================================================
# random_numbers = []
# for i in range(10):
#     r = np.random.randint(0, clf.datahandler.X_test.shape[0])
#     while r in random_numbers:
#        r = np.random.randint(0, clf.datahandler.X_test.shape[0])
#     random_numbers.append(r)
#     
# fig = plot_prob_predictions_individual(real_indices=random_numbers)    
# =============================================================================

### Plot probabilistic predictions together

In [ ]:
def _normalize_min_max(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def plot_prob_predictions_together(real_indices, normalize=False):
    legend = []
    no_of_real_spectra = len(real_indices)
    legend = [f"S{i+1}: Test spectrum no. {index}" for i, index in enumerate(real_indices)]
    legend += [f"S{i}: Test spectrum no. {real_indices[0]} with two additional peaks" for i in range(X_syn.shape[0])]
    
    X = np.vstack((clf.datahandler.X_test[real_indices], X_syn))
    y = np.vstack((clf.datahandler.y_test[real_indices], y_syn))
    prob_pred = np.vstack((clf.datahandler.prob_pred_test[real_indices], prob_pred_syn))

    energies = clf.datahandler.energies

    fontdict = {
        "size": 15,
        "family": "sans-serif"}
    fig, axs = plt.subplots(
        nrows=1, 
        ncols=len(clf.datahandler.labels)+1,
        )
    plt.subplots_adjust(
        right=3,
        wspace=0.25)

    for j, label in enumerate(clf.datahandler.labels):
        axs[j+1].set_title(label,fontdict=fontdict)
        axs[j+1].set_xlabel("Prediction",fontdict=fontdict)
        axs[j+1].set_ylabel("Counts",fontdict=fontdict)
        axs[j+1].tick_params(axis="x", labelsize=fontdict["size"])
        axs[j+1].tick_params(axis="y", labelsize=fontdict["size"])

    cmap = plt.get_cmap("tab10")
    for i in range(X.shape[0]):
        color = cmap(i)

        if normalize:
          axs[0].plot(energies, _normalize_min_max(X[i]),color=color, lw=2)
        else:
          axs[0].plot(energies, X[i],color=color, lw=2)          
        axs[0].invert_xaxis()
        axs[0].set_xlim(np.max(energies--clf.datahandler.sim_values_test["shift_x"][real_indices[0]]), np.min(energies))
        axs[0].set_xlabel("Binding energy (eV)", fontdict=fontdict)
        axs[0].set_ylabel("Intensity (arb. units)", fontdict=fontdict)
        axs[0].tick_params(axis="x", labelsize=fontdict["size"])
        axs[0].set_yticks([])

        for j, row in enumerate(prob_pred[i].transpose()):
            axs[j+1].hist(
                row,
                bins=100,
                range=(0.,1.),
                orientation="vertical",
                color=color,
                fill=True,
                linewidth=1,
                label=clf.datahandler.labels[j],
            )
            axs[j+1].set_ylim(0, 200)
    
    fig.legend(
        labels=legend,   
        loc="center right",
        bbox_to_anchor=(3.35, 0.75),
        bbox_transform=plt.gcf().transFigure, 
        fontsize=fontdict["size"]-3)
    
    #fig.tight_layout()
    
    return fig

fig = plot_prob_predictions_together(real_indices=indices, normalize=False)
fig.savefig(os.path.join(clf.logging.fig_dir, "prob_predictions_together"), bbox_inches="tight")

### Print information about the real spectra.

In [ ]:
for index in indices:
    print(f"Spectrum no. {index}")
    print("\t FWHM of Gaussian: ", clf.datahandler.sim_values_test["fwhm"][index])
    print("\t S/N: ",clf.datahandler.sim_values_test["noise"][index])

### Save data

In [ ]:
#clf.save_hyperparams()
clf.pickle_results()

## Save output of notebook

In [ ]:
from IPython.display import Javascript, display
from nbconvert import HTMLExporter

def save_notebook():
    display(Javascript("IPython.notebook.save_notebook()"),
            include=['application/javascript'])

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

import time
import os

time.sleep(20)
save_notebook()
print('Notebook saved!')
time.sleep(30)
current_file = '/content/drive/My Drive/deepxps/xpsdeeplearning/notebooks/predict_bayesian.ipynb'
output_file = os.path.join(clf.logging.log_dir,
                           'predict_bayesian_out.html')
output_HTML(current_file, output_file)
print('HTML file saved!')